In [2]:
%matplotlib
from nanorsm import *

Using matplotlib backend: Qt5Agg


In [7]:
# load elemental image data, align it with pystackreg and generate a tranform matrix

data_path = '//data//users//2023Q2//Murray_2023Q2//GAAFET_35_percent_10nm//'
sid_list = np.concatenate((np.sort(get_sid_list(['219698-219728'],3))[::-1],get_sid_list(['219620-219692'],3)))
#sid_list = get_sid_list(['218921-219056'],3)

elem = 'W_L'
num = np.size(sid_list)
file_list = []
for sid in sid_list:
    tmp = ''.join([data_path, 'output_tiff_scan2D_','{}'.format(sid), '//detsum_',elem,'_norm.tiff'])
    file_list.append(tmp)
im_stack = load_ims(file_list)

num_frame,im_row,im_col = np.shape(im_stack)

im_stack_aligned, trans_matrix = align_im_stack(im_stack) # use pystackreg

#im_stack = np.delete(im_stack,0,axis=0)
#trans_matrix = np.delete(trans_matrix,0,axis=0)

im_stack_test = interp_sub_pix(im_stack,trans_matrix) # verify the alignment is done correctly

tifffile.imwrite(''.join([data_path, 'original_rsm_100x100_nm//stack_aligned_', elem, '_100x100_nm.tif']),im_stack_test.astype(np.float32),imagej=True)

Progress:   0%|          | 0/36 [00:00<?, ?it/s]

Progress:   0%|          | 0/36 [00:00<?, ?it/s]

In [26]:
np.delete(im_stack,0,axis=0).shape

(32, 70, 40)

In [16]:
sid_list = np.concatenate((np.sort(get_sid_list(['219698-219728'],3))[::-1],get_sid_list(['219531-219588'],3)))
sid_list

array([219728, 219725, 219722, 219719, 219716, 219713, 219710, 219707,
       219704, 219701, 219698, 219531, 219534, 219537, 219540, 219543,
       219546, 219549, 219552, 219555, 219558, 219561, 219564, 219567,
       219570, 219573, 219576, 219579, 219582, 219585, 219588])

In [8]:
# if there are more elements to be aligned and stacked
elem_list = ['Cu_K','Ge_K','Ti_K']
#sid_list = get_sid_list(['219792-219888'],3)

num = np.size(sid_list)
i = 1
for elem in elem_list:
    file_list = []
    for sid in sid_list:
        tmp = ''.join([data_path, 'output_tiff_scan2D_','{}'.format(sid), '//detsum_',elem,'_norm.tiff'])
        
        file_list.append(tmp)
    im_stack = load_ims(file_list)
    
    im_stack_test = interp_sub_pix(im_stack,trans_matrix)
    imp = np.sum(im_stack_test,0)
    imp.shape
    if i == 1:
        stack = imp
        sz = np.shape(imp)
        stack = np.reshape(stack, (1,sz[0],sz[1]))
    else:
        stack = np.concatenate((stack,imp[np.newaxis,:,:]),0)
    i = i+1
    tifffile.imwrite(''.join([data_path, 'original_rsm_100x100_nm//stack_aligned_', elem, '_100x100_nm.tif']),im_stack_test.astype(np.float32),imagej=True)

Progress:   0%|          | 0/36 [00:00<?, ?it/s]

Progress:   0%|          | 0/36 [00:00<?, ?it/s]

Progress:   0%|          | 0/36 [00:00<?, ?it/s]

Progress:   0%|          | 0/36 [00:00<?, ?it/s]

Progress:   0%|          | 0/36 [00:00<?, ?it/s]

Progress:   0%|          | 0/36 [00:00<?, ?it/s]

In [19]:
plt.imshow(np.log10(tot*mask+0.1),clim=[0.1,3.5])
#tifffile.imwrite('/data/users/2023Q2/Murray_2023Q2/Substrate_35_perc_sum_all.tif',tot.astype(np.float32))

In [51]:
#sid_list = get_sid_list(['219422'],1)
tot = sum_all_h5_data_db(sid_list,'merlin2') 
plt.figure()
plt.imshow(np.log10(tot),clim=[0.25,2])

Progress:   0%|          | 0/46 [00:00<?, ?it/s]

ValueError: operands could not be broadcast together with shapes (410,318) (502,495) 

In [53]:
plt.figure()
plt.imshow(np.log10(tot),clim=[0.25,5])

/opt/conda_envs/nsls2-analysis-2021-1.2/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log10
  


In [8]:
mask = np.asarray(tifffile.imread('//data//users//2023Q2//Murray_2023Q2//nanodiffraction//mask.tif'))
plt.figure()
plt.imshow(np.log10(mask),clim=[0,5])

/opt/conda_envs/nsls2-analysis-2021-1.2/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# sum all diffraction patterns from local files to determine roi and mask
# data_path = '//data//home//home//hyan//export//scan_180000//'
#prefix = 'scan_'
#postfix = '_merlin1.h5'

#data_file_list = create_file_list(data_path, prefix, postfix, sid_list)
#tot = sum_all_h5_data(data_file_list)
#plt.imshow(np.log10(tot+0.1),clim=[1,5])

In [ ]:
# determine from tot image
#mask = np.ones(tot.shape)

#plt.imshow(np.log10(tot*mask+0.1))
#roi = [185,105,160,130]

In [4]:
# Load, normalize, align, and assemble diffraction data through databroker
# diff_data is the 5D dataset aligned; make sure your machine has enough memory
roi = [180,5,120,250]
#mask = np.asarray(tifffile.imread('//data//users//2023Q2//Murray_2023Q2//nanodiffraction//mask.tif'))

scan_row = im_row
scan_col = im_col

diff_data = load_h5_data_db(sid_list,det='merlin2', mon='sclr1_ch4', roi=roi)

sz = diff_data.shape
diff_data = np.reshape(diff_data,(sz[0],scan_row,scan_col,sz[2],sz[3]))
diff_data = interp_sub_pix(diff_data,trans_matrix)

Progress:   0%|          | 0/46 [00:00<?, ?it/s]

Total scan points: 11000; raw image row: 410; raw image col: 318
Total scan points: 11000; data image row: 120; data image col: 250


Progress:   0%|          | 0/46 [00:00<?, ?it/s]

In [5]:
plt.figure()
plt.imshow(diff_data[0,20,20,:,:])

In [43]:
diff_data[:,:,:,66,146] = 0
diff_data[:,:,:,188,107] = 0

In [ ]:
# Load, normalize, align, and assemble diffraction data 
# diff_data is the 5D dataset aligned; make sure your machine has enough memory
'''
data_path = '//data//home//home//hyan//export//scan_180000//'
prefix = 'scan_'
postfix = '_merlin1.h5'
scan_row = 100
scan_col = 100

data_file_list = create_file_list(data_path, prefix, postfix, sid_list)
diff_data = load_h5_data(data_file_list,roi=roi,mask=mask)

postfix = '_merlin1.txt'
mon_file_list = create_file_list(data_path, prefix, postfix, sid_list)

mon_data = load_scaler_data(mon_file_list,['sclr1_ch4'])

# First scaler reading of each scan is zero; refill with first non-zero reading
avg = np.mean(mon_data[mon_data != 0])
mon_data[mon_data==0] = avg 

sz = diff_data.shape

diff_data = (diff_data/mon_data[:,np.newaxis,np.newaxis])

diff_data = np.reshape(diff_data,(sz[0],scan_row,scan_col,sz[2],sz[3]))
diff_data = interp_sub_pix(diff_data,trans_matrix)
'''

In [6]:
# transform to cartesian crystal coordinates (z along hkl and x is the rocking direction)

energy = 12
delta = 0
gamma = 44.7
num_angle = 46
th_step = 0.02
pix = 55
det_dist = 4e5
offset = [0,0]
data_store = 'reduced' # this will reduce the data stored. If use 'full' it can be over 100G
save_path = ''.join([data_path,'original_rsm_blanket_film//'])

# generate an object of the RSM class
rsm = RSM(diff_data,energy,delta,gamma,num_angle,th_step,pix,det_dist,offset)
# transform from detector coordinates to crystal coordinates
rsm.calcRSM('cryst',data_store)
# calculate strain
# 'com', center of mass, is a simple algorithm to calculate the strain. Note: There is an abitrary offset
#rsm.calcSTRAIN('com')  
# show results
#rsm.disp()
# save results
#rsm.save(save_path)
# also save the entire object
save_file = ''.join([save_path,'rsm_35_blanket_film.obj'])
pickle.dump(rsm, open(save_file,'wb'),protocol = 4)

Progress:   0%|          | 0/100 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '//data//users//2023Q2//Murray_2023Q2//Blanket_film_overnight//original_rsm_blanket_film//rsm_35_blanket_film.obj'

In [7]:
save_file = ''.join([data_path,'rsm_35_blanket_film.obj'])
pickle.dump(rsm, open(save_file,'wb'),protocol = 4)

In [ ]:
plt.figure()
plt.imshow(np.log10(rsm.qxz_data[30,70,:,:].T+1e-7),clim=[-6,0])
plt.show()

In [ ]:
data_path = '//data//users//2020Q3//Ajith_2020Q3//LPE_FeS2_P4_diff//'
save_file = ''.join([data_path,'rsm.obj'])
rsm = pickle.load(open(save_file,'rb'))
rsm.disp()

In [44]:
plt.figure()
plt.imshow(np.squeeze(np.sum(np.sum(diff_data[-2,:,:,:,:],axis=0),0)))


In [8]:
%matplotlib
interactive_map(['Gu_K','Ge_K','Ti_K'],stack,'rsm',np.log10(np.swapaxes(rsm.qxz_data,2,3)+1e-6),cmap='jet')

Using matplotlib backend: Qt5Agg


In [76]:
plt.figure()
plt.imshow(np.log10(np.swapaxes(rsm.qxz_data,2,3)[11,25,:,:]+1e-7),cmap='jet',clim=[-4.8,-2.5])

In [ ]:
# interactive figure

%matplotlib qt

# load fluorescence image stacks
# insert your own code accordingly

sum_Ge = np.sum(Ge,0)
sum_W = np.sum(W,0)
sum_Cu = np.sum(Cu,0)

cmap = 'viridis'
color = 'red'
eps = 1e-5
def onclick(event):
    global row, col
    col, row = event.xdata, event.ydata
    if col is not None and row is not None:
        row = int(np.round(row))
        col = int(np.round(col))
        ax0.clear()
        ax0.imshow(sum_Ge,cmap=cmap)
        ax0.plot(col,row,marker='o',markersize=2, color=color)
        ax0.set_title('Ge')
        ax1.clear()
        ax1.imshow(sum_W,cmap=cmap)
        ax1.plot(col,row,marker='o',markersize=2, color=color)
        ax1.set_title('W')
        ax2.clear()
        ax2.imshow(sum_Cu,cmap=cmap)
        ax2.plot(col,row,marker='o',markersize=2, color=color)
        ax2.set_title('Cu')
        ax3.clear()
        ax3.imshow(rsm.tot,cmap=cmap)
        ax3.plot(col,row,marker='o',markersize=2, color=color)
        ax3.set_title('tot')
        ax4.clear()
        ax4.imshow(np.log10(rsm.qxz_data[row,col,:,:].T+eps),cmap=cmap)
        ax4.set_title('RSM')
        fig.canvas.draw_idle()
    
    return

fig = plt.figure()
spec = fig.add_gridspec(2,3)
#fig, ax = plt.subplots(2,1)
ax0 = fig.add_subplot(spec[0,0])
ax0.imshow(clean_Ge,cmap=cmap)
ax0.set_title('Ge')

ax1 = fig.add_subplot(spec[0,1])
ax1.imshow(sum_W,cmap=cmap)
ax1.set_title('W')

ax2 = fig.add_subplot(spec[0,2])
ax2.imshow(sum_Cu,cmap=cmap)
ax2.set_title('Cu')

ax3 = fig.add_subplot(spec[1,0])
ax3.imshow(rsm.tot,cmap=cmap)
ax3.set_title('tot')

ax4 = fig.add_subplot(spec[1,1:3])
ax4.imshow(np.log10(rsm.qxz_data[0,0,:,:].T+eps),cmap=cmap)
ax4.set_title('RSM')
plt.tight_layout()

# Call click func
cid = fig.canvas.mpl_connect('button_press_event', onclick)

#plt.show()

In [ ]:
# create movie
import matplotlib
import matplotlib.animation as manimation 
matplotlib.use('AGG') 
def update_fig(row,col):

    #col = col + i
    ax0.clear()
    ax0.imshow(sum_Ge,cmap=cmap)
    ax0.plot(col,row,marker='o',markersize=2, color=color)
    ax0.set_title('Ge')
    ax1.clear()
    ax1.imshow(sum_W,cmap=cmap)
    ax1.plot(col,row,marker='o',markersize=2, color=color)
    ax1.set_title('W')
    ax2.clear()
    ax2.imshow(sum_Cu,cmap=cmap)
    ax2.plot(col,row,marker='o',markersize=2, color=color)
    ax2.set_title('Cu')
    ax3.clear()
    ax3.imshow(rsm.tot,cmap=cmap)
    ax3.plot(col,row,marker='o',markersize=2, color=color)
    ax3.set_title('tot')
    ax4.clear()
    ax4.imshow(np.log10(rsm.qxz_data[row,col,:,:].T+eps),cmap=cmap)
    ax4.set_title('RSM')
    fig.canvas.draw_idle()
    return  
     
def init():
    fig = plt.figure()
    spec = fig.add_gridspec(2,3)
    
    ax0 = fig.add_subplot(spec[0,0])
    ax0.imshow(clean_Ge,cmap=cmap)
    ax0.set_title('Ge')

    ax1 = fig.add_subplot(spec[0,1])
    ax1.imshow(sum_W,cmap=cmap)
    ax1.set_title('W')

    ax2 = fig.add_subplot(spec[0,2])
    ax2.imshow(sum_Cu,cmap=cmap)
    ax2.set_title('Cu')

    ax3 = fig.add_subplot(spec[1,0])
    ax3.imshow(rsm.tot,cmap=cmap)
    ax3.set_title('tot')

    ax4 = fig.add_subplot(spec[1,1:3])
    ax4.imshow(np.log10(rsm.qxz_data[0,0,:,:].T+eps),cmap=cmap)
    ax4.set_title('RSM')
    plt.tight_layout()
    return  

FFMpegWriter = manimation.writers['ffmpeg']
metadata = dict(title='Movie', artist='hyan',
                comment='RSM variation across GAAFET')
writer = FFMpegWriter(fps=5, metadata=metadata)

init()

with writer.saving(fig, 'myfile.mp4', dpi=100):
    writer.grab_frame()
    row = 21
    col = 5
    for j in range(44):
        update_fig(row,col+j)
        writer.grab_frame()
    row = 2
    col = 28
    for j in range(40):
        update_fig(row+j,col)
        writer.grab_frame()
    #writer.finish()
 